In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/c2d4u.tea

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-25 16:45:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.43MB/s    in 0.2s    

2022-09-25 16:45:28 (5.43 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [ ]:
# Function to use for columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes >= 20
vine_df_vote = vine_df.filter(col('total_votes')>=20)
vine_df_vote.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [ ]:
# Calculate % of helpful votes
import pyspark.sql.functions as F

vine_df_vote = vine_df_vote.withColumn('%_helpful_vote', F.col('helpful_votes')/F.col('total_votes')*100)

In [ ]:
# Retrieve helpful votes
vine_df_helpful = vine_df_vote.filter(col('%_helpful_vote')>=50)
vine_df_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   %_helpful_vote|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [ ]:
# Reviews written as part of Vine program
vine_df_helpful_paid = vine_df_helpful.filter(col('vine')=='Y')
vine_df_helpful_paid.show()

+---------+-----------+-------------+-----------+----+-----------------+--------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|%_helpful_vote|
+---------+-----------+-------------+-----------+----+-----------------+--------------+
+---------+-----------+-------------+-----------+----+-----------------+--------------+



In [ ]:
# Unpaid reviews
vine_df_helpful_unpaid = vine_df_helpful.filter(col('vine')=='N')
vine_df_helpful_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   %_helpful_vote|
+--------------+-----------+-------------+-----------+----+-----------------+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|             80.0|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|67.74193548387096|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|96.42857142857143|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|            100.0|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|87.29641693811075|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|88.88888888888889|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|95.83333333333334|
|R2JSWNTAO83X3V|          5|          14

In [ ]:
# Count reviews
count_vine_helpful_unpaid = vine_df_helpful_unpaid.count()
count_vine_helpful_unpaid_five = vine_df_helpful_unpaid.filter(col('star_rating')=='5').count()

In [ ]:
# Calculate % of 5-star reviews
p_vine_df_helpful_unpaid_five = count_vine_helpful_unpaid_five/count_vine_helpful_unpaid*100

In [ ]:
# Create summary
dataframe = spark.createDataFrame([    
    ('Unpaid', count_vine_helpful_unpaid, count_vine_helpful_unpaid_five, round(p_vine_df_helpful_unpaid_five, 1))
], ['Review Type', 'Total Helpful Reviews', '5-Star Reviews', '5-Star Reviews (%)'])
dataframe.show()

+-----------+---------------------+--------------+------------------+
|Review Type|Total Helpful Reviews|5-Star Reviews|5-Star Reviews (%)|
+-----------+---------------------+--------------+------------------+
|     Unpaid|               403807|        242889|              60.1|
+-----------+---------------------+--------------+------------------+



In [ ]:
# Paid review in original table
df.filter(col('vine')=="Y").count()

2